# Velocidade terminal

**Fonte:** Operações Unitárias em Sistemas Particulados e Fluidomecânicos por Marco Aurélio Cremasco

In [1]:
def aviso(string):
    import warnings
    warnings.filterwarnings(string)
    pass

# Método iterativo

Função: **rey_p**
\begin{align}
Re_p &= \frac{\rho d_p v_t}{\mu}
\end{align}
Função: **vel_term**
\begin{align}
v_t &= \sqrt{\frac{4}{3} \frac{d_p (\rho_s - \rho) g}{\rho C_D}}
\end{align}

In [2]:
def vel_term(arrasto, diametro, densidade_solido, densidade_fluido=997, gravidade=9.81) -> float:
    return ( 4/3 * diametro*(densidade_solido - densidade_fluido)*gravidade / (densidade_fluido*arrasto) )**(1/2)

def rey_p(velocidade_terminal, diametro, viscosidade_fluido=0.00089008, densidade_fluido=997) -> float:
    return densidade_fluido*diametro*velocidade_terminal/viscosidade_fluido

arrasto = {'stokes': lambda reynolds: 24/reynolds,
           'allen': lambda reynolds: 18.5/reynolds**0.6, 
           'klyachko': lambda reynolds: 24/reynolds + 4*reynolds**(-1/3),
           'langmuir': lambda reynolds: 24/reynolds * (1 + 0.197*reynolds**0.63 + 0.0026*reynolds**1.39),
           'newton': lambda reynolds: 0.44,
           'turbulento': lambda reynolds: 0.20}

def velocidade_terminal(y0, diametro, densidade_solido, viscosidade_fluido=0.00089008, densidade_fluido=997, gravidade=9.81, metodo='todos', tolerancia=1.49012e-08) -> dict:
    #METODOS: 'stokes', 'allen', 'klyachko', 'langmuir', 'newton', 'turbulento', 'todos'
    from scipy.optimize import fsolve
    from numpy import array
    def func(y):
        re = rey_p(y, diametro, viscosidade_fluido=viscosidade_fluido, densidade_fluido=densidade_fluido)
        cd = arrasto[auxiliar](re)
        vt = vel_term(cd, diametro, densidade_solido, densidade_fluido, gravidade)
        return y - vt
    
    if metodo == 'todos':
        resultados = {}
        for auxiliar in arrasto:
            if auxiliar not in ('newton', 'turbulento'):
                resp = fsolve(func, y0, xtol=tolerancia)
                vt = resp[0]
                re = rey_p(vt, diametro, viscosidade_fluido=viscosidade_fluido, densidade_fluido=densidade_fluido)
            else:
                cd = arrasto[auxiliar](0)
                vt = vel_term(cd, diametro, densidade_solido, densidade_fluido, gravidade)
                re = rey_p(vt, diametro, viscosidade_fluido=viscosidade_fluido, densidade_fluido=densidade_fluido)
            resultados[auxiliar] = array([vt, re])
 
    elif metodo not in ('newton', 'turbulento'):
        auxiliar = metodo
        resp = fsolve(func, y0, xtol=tolerancia)
        vt = resp[0]
        re = rey_p(vt, diametro, viscosidade_fluido=viscosidade_fluido, densidade_fluido=densidade_fluido)
        resultados = {metodo: array([vt, re])}
    else:
        cd = arrasto[metodo](0)
        vt = vel_term(cd, diametro, densidade_solido, densidade_fluido, gravidade)
        re = rey_p(vt, diametro, viscosidade_fluido=viscosidade_fluido, densidade_fluido=densidade_fluido)
        resultados = {metodo: array([vt, re])}
        
    return resultados
    
def diametro(y0, vel_terminal, densidade_solido, viscosidade_fluido=0.00089008, densidade_fluido=997, gravidade=9.81, metodo='todos', tolerancia=1.49012e-08) -> dict:
    #METODOS: 'stokes', 'allen', 'klyachko', 'langmuir', 'newton', 'turbulento', 'todos'
    from scipy.optimize import fsolve
    from numpy import array
    def func(y):
        re = rey_p(vel_terminal, y, viscosidade_fluido=viscosidade_fluido, densidade_fluido=densidade_fluido)
        cd = arrasto[auxiliar](re)
        vt = vel_term(cd, y, densidade_solido, densidade_fluido, gravidade)
        return vel_terminal - vt

    if metodo == 'todos':
        resultados = {}
        for auxiliar in arrasto:
            if auxiliar not in ('newton', 'turbulento'):
                resp = fsolve(func, y0, xtol=tolerancia)
                di = resp[0]
                re = rey_p(vel_terminal, di, viscosidade_fluido=viscosidade_fluido, densidade_fluido=densidade_fluido)
            else:
                resp = fsolve(func, y0, xtol=tolerancia)
                di = resp[0]
                re = rey_p(vel_terminal, di, viscosidade_fluido=viscosidade_fluido, densidade_fluido=densidade_fluido)
            resultados[auxiliar] = array([di, re])

    elif metodo not in ('newton', 'turbulento'):
        auxiliar = metodo
        resp = fsolve(func, y0, xtol=tolerancia)
        di = resp[0]
        re = rey_p(vel_terminal, di, viscosidade_fluido=viscosidade_fluido, densidade_fluido=densidade_fluido)
        resultados = {metodo: array([di, re])}
    else:
        resp = fsolve(func, y0, xtol=tolerancia)
        di = resp[0]
        re = rey_p(vel_terminal, di, viscosidade_fluido=viscosidade_fluido, densidade_fluido=densidade_fluido)
        resultados = {metodo: array([di, re])}
        
    return resultados

# Método exato de Stokes
**Função exata de Stokes**
\begin{align}
v_t &= \frac{d_p^2 (\rho_s - \rho) g}{18 \mu}
\end{align}

In [3]:
def stokes(diametro, vel_terminal, densidade_solido, viscosidade_fluido=0.00089008, densidade_fluido=997, gravidade=9.81) -> list:
    #ESSA FUNÇÃO SÓ PERMITE UMA VÁRIAVEL DE CADA VEZ, CASO SEJAM COLOCADOS DOIS X A FUNÇÃO IRÁ FALHAR (isso é intencional)
    from sympy import symbols, solveset
    variaveis = [diametro, vel_terminal, densidade_solido, viscosidade_fluido, densidade_fluido, gravidade]
              # [0, 1, 2, 3, 4, 5] ordem dos valores da lista de variaveis
    x = symbols('x')
    
    cont = 0
    for i, j in enumerate(variaveis): #procura qual é a variável e qual a sua posição na lista de variáveis
        if j == 'x': variaveis[i], cont = x, cont+1
    
    if cont == 1: #checa se existe apenas uma variável
        function =  variaveis[1] - variaveis[0]**2 * (variaveis[2] - variaveis[4]) * variaveis[5]/(18 * variaveis[3])

        return list(solveset(function, x))
    else: #ter mais ou menos de uma variáveis
        print('Equação sem variável ou com mais de uma.')
        pass

In [4]:
aviso('default')
#aviso('ignore')

## Calculando a solução exata em Stokes para usar de comparativo com as outras funções
A solução de stokes pode indicar um bom chute inicial para as outras

obs.: Usando os valores da questão 2

In [5]:
stokes('x', 0.04, 7500)

[-0.000100228074890817, 0.000100228074890817]

In [6]:
rey_p(0.04, stokes('x', 0.04, 7500)[1])

4.49071502184724

In [7]:
arrasto['stokes'](rey_p(0.04, stokes('x', 0.04, 7500)[1]))

5.34436050456118

In [8]:
vel_term(arrasto['stokes'](rey_p(0.04, stokes('x', 0.04, 7500)[1])),
         stokes('x', 0.04, 7500)[1], 7500)

0.0400000000000000

## Testando a função 'velocidade_terminal'
Critérios:
 1. Chute inicial
 2. Tolerância
 3. Métodos

In [9]:
stokes(7e-4, 'x', 2650)[0], rey_p(stokes(7e-4, 'x', 2650)[0], 7e-4) #metodo exato de stokes

(0.495948285547366, 388.866515912622)

In [10]:
velocidade_terminal(1, 7e-4, 2650, tolerancia=1e-9) #chute inicial válido

{'stokes': array([  0.49594829, 388.86651591]),
 'allen': array([ 0.10870159, 85.23148289]),
 'klyachko': array([ 0.11432161, 89.63806638]),
 'langmuir': array([ 0.09728476, 76.27970115]),
 'newton': array([  0.18574426, 145.63962671]),
 'turbulento': array([  0.27550326, 216.01847585])}

In [11]:
#DIFERENÇA DA SOLUÇÃO EXATA E DA NUMÉRICA
from numpy import array
array([stokes(7e-4, 'x', 2650)[0], rey_p(stokes(7e-4, 'x', 2650)[0], 7e-4)]) -\
    array(velocidade_terminal(1, 7e-4, 2650, metodo='stokes', tolerancia=1e-9)['stokes'])

array([-1.11022302462516e-16, -5.68434188608080e-14], dtype=object)

In [12]:
velocidade_terminal(0.0001, 7e-4, 2650, tolerancia=1e-8) #chute inicial baixo demais

C:\Users\rafae\AppData\Local\Temp/ipykernel_19464/937068166.py:2: RuntimeWarning: invalid value encountered in sqrt
  return ( 4/3 * diametro*(densidade_solido - densidade_fluido)*gravidade / (densidade_fluido*arrasto) )**(1/2)
C:\Users\rafae\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
C:\Users\rafae\AppData\Local\Temp/ipykernel_19464/937068166.py:8: RuntimeWarning: invalid value encountered in power
  'allen': lambda reynolds: 18.5/reynolds**0.6,
C:\Users\rafae\AppData\Local\Temp/ipykernel_19464/937068166.py:9: RuntimeWarning: invalid value encountered in power
  'klyachko': lambda reynolds: 24/reynolds + 4*reynolds**(-1/3),
C:\Users\rafae\AppData\Local\Temp/ipykernel_19464/937068166.py:10: RuntimeWarning: invalid value encountered in power
  'langmuir': lambda reynolds: 24/reynolds * (1 + 0.197*reynolds**0.63 + 

{'stokes': array([0.0001    , 0.07840868]),
 'allen': array([0.0001    , 0.07840868]),
 'klyachko': array([0.0001    , 0.07840868]),
 'langmuir': array([0.0001    , 0.07840868]),
 'newton': array([  0.18574426, 145.63962671]),
 'turbulento': array([  0.27550326, 216.01847585])}

In [13]:
velocidade_terminal(1, 7e-4, 2650, tolerancia=1e-4) #tolerância ainda válida

{'stokes': array([  0.49594829, 388.86651594]),
 'allen': array([ 0.10870159, 85.23148504]),
 'klyachko': array([ 0.11432161, 89.63806672]),
 'langmuir': array([ 0.09728476, 76.27970116]),
 'newton': array([  0.18574426, 145.63962671]),
 'turbulento': array([  0.27550326, 216.01847585])}

In [14]:
velocidade_terminal(1, 7e-4, 2650, tolerancia=1e-0) #tolerância alterou significativamente os valores

{'stokes': array([  0.50259876, 394.08106728]),
 'allen': array([ 0.11727179, 91.95126664]),
 'klyachko': array([ 0.12013175, 94.1937231 ]),
 'langmuir': array([ 0.09972099, 78.18991623]),
 'newton': array([  0.18574426, 145.63962671]),
 'turbulento': array([  0.27550326, 216.01847585])}

In [15]:
velocidade_terminal(100, 7e-4, 2650, metodo='allen', tolerancia=1e-7) #chute inicial válido

{'allen': array([ 0.10870159, 85.23148289])}

In [16]:
velocidade_terminal(1, 7e-4, 2650, metodo='klyachko', tolerancia=1e-7) #chute inicial válido

{'klyachko': array([ 0.11432161, 89.63806638])}

In [17]:
velocidade_terminal(3, 7e-4, 2650, metodo='klyachko', tolerancia=1e-7) #chute inicial válido

{'klyachko': array([ 0.11432161, 89.63806638])}

In [18]:
velocidade_terminal(0.025, 7e-4, 2650, metodo='langmuir', tolerancia=1e-7) #chute inicial válido

{'langmuir': array([ 0.09728476, 76.27970115])}

In [19]:
velocidade_terminal(0.020, 7e-4, 2650, metodo='langmuir', tolerancia=1e-7) #chute inicial baixo demais

C:\Users\rafae\AppData\Local\Temp/ipykernel_19464/937068166.py:10: RuntimeWarning: invalid value encountered in power
  'langmuir': lambda reynolds: 24/reynolds * (1 + 0.197*reynolds**0.63 + 0.0026*reynolds**1.39),


{'langmuir': array([ 0.02      , 15.68173647])}

In [20]:
resolucao = velocidade_terminal(1., 7e-4, 2650, metodo='langmuir', tolerancia=1e-9)
v_terminal = resolucao['langmuir'][0]
print(f'Para um diâmetro de 700 micrometros, a velocidade terminal é de {round(v_terminal*100, 2)} cm/s')

Para um diâmetro de 700 micrometros, a velocidade terminal é de 9.73 cm/s


## Testando a função 'diametro'
Critérios:
 1. Chute inicial
 2. Tolerância
 3. Métodos

In [21]:
diametro(0.1, 0.04, 7500, metodo='stokes', tolerancia=1e-9) #raiz positiva (chute positivo)

{'stokes': array([1.00228075e-04, 4.49071502e+00])}

In [22]:
diametro(-0.1, 0.04, 7500, metodo='stokes', tolerancia=1e-9) #raiz negativa (chute negativo)

{'stokes': array([-1.00228075e-04, -4.49071502e+00])}

In [23]:
diametro(1e-6, 0.04, 7500, metodo='todos', tolerancia=1e-9)

{'stokes': array([1.00228075e-04, 4.49071502e+00]),
 'allen': array([1.23999110e-04, 5.55577532e+00]),
 'klyachko': array([1.23643029e-04, 5.53982115e+00]),
 'langmuir': array([1.27574235e-04, 5.71595868e+00]),
 'newton': array([8.25175489e-06, 3.69719559e-01]),
 'turbulento': array([3.75079768e-06, 1.68054345e-01])}

In [24]:
diametro(0.001, 0.04, 7500, metodo='todos', tolerancia=1e-9) #chute inicial muito alto

C:\Users\rafae\AppData\Local\Temp/ipykernel_19464/937068166.py:8: RuntimeWarning: invalid value encountered in power
  'allen': lambda reynolds: 18.5/reynolds**0.6,
C:\Users\rafae\AppData\Local\Temp/ipykernel_19464/937068166.py:9: RuntimeWarning: invalid value encountered in power
  'klyachko': lambda reynolds: 24/reynolds + 4*reynolds**(-1/3),
C:\Users\rafae\AppData\Local\Temp/ipykernel_19464/937068166.py:10: RuntimeWarning: invalid value encountered in power
  'langmuir': lambda reynolds: 24/reynolds * (1 + 0.197*reynolds**0.63 + 0.0026*reynolds**1.39),
C:\Users\rafae\AppData\Local\Temp/ipykernel_19464/937068166.py:2: RuntimeWarning: invalid value encountered in sqrt
  return ( 4/3 * diametro*(densidade_solido - densidade_fluido)*gravidade / (densidade_fluido*arrasto) )**(1/2)


{'stokes': array([1.00228075e-04, 4.49071502e+00]),
 'allen': array([1.00000000e-03, 4.48049614e+01]),
 'klyachko': array([1.00000000e-03, 4.48049614e+01]),
 'langmuir': array([1.00000000e-03, 4.48049614e+01]),
 'newton': array([1.00000000e-03, 4.48049614e+01]),
 'turbulento': array([1.00000000e-03, 4.48049614e+01])}

In [25]:
resolucao = diametro(1e-6, 0.04, 7500, metodo='langmuir', tolerancia=1e-9)
diam = resolucao['langmuir'][0]
print(f'Para uma velocidade terminal de 4 cm/s, o diâmetro deve ser de {round(diam*1e+6, 2)} micrometros')

Para uma velocidade terminal de 4 cm/s, o diâmetro deve ser de 127.57 micrometros


# Método de Massarani
(COELHO e MASSARANI, 1996)

**Função:** massarani_d
\begin{align}
d_i &= Re \frac{\mu}{\rho v_t}
\end{align}

Usa as variáveis n, Cd_Re, k1, k2, re2

O primeiro retorno é o valor do diâmetro e o segundo é o número de reynolds

Sobre *esf_perf*:
1. True: n = 0.88
2. False: n = 1.3

**Função:** massarani_v
\begin{align}
v_t &= Re \frac{\mu}{\rho d_i}
\end{align}

Usa as variáveis n, CdRe2, k1, k2, re1

O primeiro retorno é o valor da velocidade terminal e o segundo é o número de reynolds

Sobre *esf_perf*:
1. True: n = 0.95
2. False: n = 1.2

**Observações:**

Para os casos em que a particula não é uma esfera perfeita, limita-se a esfericidades entre **]0.65, 1]** e reynolds menores ou iguais à **5e+4**

O argumento *esf_perf* significa esfera perfeita e deve vim acoplado ao termo *esfericidade* valendo 1, ou então será desconsiderada a informação de particula perfeitamente esférica.

**Variável:** CdRe2
\begin{align}
C_D Re^2 &= \frac{4}{3} \frac{\rho (\rho_p - \rho) b d_p^3}{\mu^2}
\end{align}

**Variável:** Cd_Re
\begin{align}
C_D/Re &= \frac{4}{3} \frac{(\rho_p - \rho) \mu b}{\rho^2 v_t^2}
\end{align}

**Variável:** k1
\begin{align}
K_1 &= 0.843 \log_{10}(\phi/0.065)
\end{align}

**Variável:** k2
\begin{align}
K_2 &= 5.31 - 4.88\phi
\end{align}

**Variável:** re1
\begin{align}
Re &= \left[ \left(\frac{K_1 C_D Re^2}{24} \right)^{-n} + \left(\frac{C_D Re^2}{K_2} \right)^{-n/2} \right]^{-1/n}
\end{align}

**Variável:** re2
\begin{align}
Re &= \left[ \left(\frac{24}{K_1 C_D/Re} \right)^{n/2} + \left(\frac{K_2}{C_D/Re} \right)^{n} \right]^{1/n}
\end{align}

In [1]:
def massarani_d(vel_terminal, densidade_solido, viscosidade_fluido=0.00089008, densidade_fluido=997, campo_externo=9.81, esfericidade=1., esf_perf=False):
    from numpy import log10, array
    n = 0.88 if (esfericidade == 1 and esf_perf == True) else 1.3
    Cd_Re = 4/3 * ((densidade_solido - densidade_fluido)*campo_externo*viscosidade_fluido)/(densidade_fluido**2 * vel_terminal**3) 
    k1 = 0.843*log10(esfericidade/0.065)
    k2 = 5.31 - 4.88*esfericidade
    re2 = ( (24/(k1*Cd_Re))**(n/2) + (k2/Cd_Re)**(n) )**(1/n)
    di = re2 * (viscosidade_fluido/(densidade_fluido*vel_terminal))
    return array([di, re2])
    
def massarani_v(diametro, densidade_solido, viscosidade_fluido=0.00089008, densidade_fluido=997, campo_externo=9.81, esfericidade=1., esf_perf=False):
    from numpy import log10, array
    n = 0.95 if (esfericidade == 1 and esf_perf == True) else 1.2
    CdRe2 = 4/3 * (densidade_fluido*(densidade_solido - densidade_fluido)*campo_externo*diametro**3)/viscosidade_fluido**2
    k1 = 0.843*log10(esfericidade/0.065)
    k2 = 5.31 - 4.88*esfericidade
    re1 = ( (k1*CdRe2/24)**-n + (CdRe2/k2)**-(n/2) )**(-1/n)
    vt = re1 * (viscosidade_fluido/(densidade_fluido*diametro))
    return array([vt, re1])

## Testando para a velocidade terminal 
**Função:** massarani_v

**Problema exemplo do livro do Cremasco**

In [2]:
massarani_v(250e-6, 2430, densidade_fluido=995.7, viscosidade_fluido=0.83e-6*995.7, esfericidade=1)

array([ 0.04209656, 12.67968825])

In [3]:
massarani_v(250e-6, 2430, densidade_fluido=995.7, viscosidade_fluido=0.83e-6*995.7, esfericidade=1, esf_perf=True)

array([ 0.03651311, 10.99792362])

In [39]:
vtm, rey = massarani_v(250e-6, 2430, densidade_fluido=995.7, viscosidade_fluido=0.83e-6*995.7, esfericidade=1, esf_perf=True)

print(f'Para um diâmetro de 250 micrometros e densidade de 2430 kg/m3, em água, a velocidade terminal é de {round(vtm*100, 2)} cm/s')

Para um diâmetro de 250 micrometros e densidade de 2430 kg/m3, em água, a velocidade terminal é de 3.65 cm/s


**Problema de velocidade terminal anterior**

In [29]:
massarani_v(7e-4, 2650, esfericidade=1) #METODO DE MASSARANI

array([  0.14986523, 117.50735125])

In [30]:
velocidade_terminal(1, 7e-4, 2650, tolerancia=1e-9) #METODO ITERATIVO

{'stokes': array([  0.49594829, 388.86651591]),
 'allen': array([ 0.10870159, 85.23148289]),
 'klyachko': array([ 0.11432161, 89.63806638]),
 'langmuir': array([ 0.09728476, 76.27970115]),
 'newton': array([  0.18574426, 145.63962671]),
 'turbulento': array([  0.27550326, 216.01847585])}

In [31]:
stokes(7e-4, 'x', 2650)[0], rey_p(stokes(7e-4, 'x', 2650)[0], 7e-4) #METODO EXATO DE STOKES

(0.495948285547366, 388.866515912622)

## Testando para o diâmetro
**Função:** massarani_d

In [6]:
diam = massarani_d(0.04, 7500, esfericidade=1)[0] #METODO DE MASSARANI

print(f'Para uma velocidade terminal de 4 cm/s, o diâmetro deve ser de {round(diam*1e+6, 2)} micrometros')

Para uma velocidade terminal de 4 cm/s, o diâmetro deve ser de 103.09 micrometros


In [7]:
massarani_d(0.04, 7500, esfericidade=1, esf_perf=True)

array([1.12680784e-04, 5.04865817e+00])

In [68]:
dia, rey = massarani_d(0.04, 7500, esfericidade=1, esf_perf=True)

print(f'Para uma velocidade terminal de 4 cm/s e densidade de 7500 kg/m3, em água, o diâmetro é de {round(dia*1e+6, 2)} micrometros')

Para uma velocidade terminal de 4 cm/s e densidade de 7500 kg/m3, em água, o diâmetro é de 763.48 micrometros


In [70]:
rey

34.20764687184174

In [71]:
massarani_v(dia, 7500, esfericidade=1, esf_perf=True)

array([  0.32603548, 278.82266652])

In [72]:
velocidade_terminal(1, dia, 7500)

{'stokes': array([   2.32100139, 1984.89989685]),
 'allen': array([  0.31930585, 273.06754603]),
 'klyachko': array([  0.3001803, 256.7115417]),
 'langmuir': array([  0.22511944, 192.52015886]),
 'newton': array([  0.38475598, 329.03991718]),
 'turbulento': array([  0.57068534, 488.0450672 ])}

In [34]:
diametro(1e-6, 0.04, 7500, metodo='todos', tolerancia=1e-9) #METODO ITERATIVO

{'stokes': array([1.00228075e-04, 4.49071502e+00]),
 'allen': array([1.23999110e-04, 5.55577532e+00]),
 'klyachko': array([1.23643029e-04, 5.53982115e+00]),
 'langmuir': array([1.27574235e-04, 5.71595868e+00]),
 'newton': array([8.25175489e-06, 3.69719559e-01]),
 'turbulento': array([3.75079768e-06, 1.68054345e-01])}

In [35]:
stokes('x', 0.04, 7500)[1], rey_p(0.04, stokes('x', 0.04, 7500)[1]) #METODO EXATO DE STOKES

(0.000100228074890817, 4.49071502184724)

In [76]:
#teste (fazendo o procedimento inverso para achar o diâmetro)
def func(di):
    return 0.04 - massarani_v(di, 7500, esf_perf=True)[0]

from scipy.optimize import fsolve
dia, rey = massarani_d(0.04, 7500, esfericidade=1, esf_perf=True)
dia2 = fsolve(func, 1e-7)

massarani_v(dia2[0], 7500, esf_perf=True), massarani_v(dia, 7500, esf_perf=True), dia2[0], dia

(array([0.04      , 5.33045809]),
 array([  0.32603548, 278.82266652]),
 0.00011897026416611484,
 0.0007634789951777556)